$$
\newcommand{\mymat}[1]{
\left[
\begin{array}{rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr}
#1
\end{array}
\right]
}
\newcommand{\myaug}[1]{
\left[
\begin{array}{rrr|r}
#1
\end{array}
\right]
}
\newcommand{\myp}[1]{\left( #1 \right)}
\newcommand{\myb}[1]{\left[ #1 \right]}
\newcommand{\myv}[1]{\left< #1 \right>}
\newcommand{\mys}[1]{\left\{ #1 \right\}}
\newcommand{\myab}[1]{\left| #1 \right|}
\newcommand{\bx}{{\bf x}}
\newcommand{\by}{{\bf y}}
\newcommand{\bu}{{\bf u}}
\newcommand{\bv}{{\bf v}}
\newcommand{\be}{{\bf e}}
$$

#Lecture 7

***

<br>

In this lecture we'll examine the computational costs associated with the elementry matrix and vector operations we've studied so far in the course.  Keep in mind though that raw computational cost is not the only thing that determines the performance of an algorithm.  In the next lecture we'll look at how memory access of an algorithm can play a large role in the overall performance of an implementation.  

Recall that a real $m \times n$ matrix $A$ has $m$ rows and $n$ columns, and we refer to the entry in the $i^{\textrm{th}}$ row and $j^{\textrm{th}}$ column of the matrix as $a_{ij}$. 

<br>

$$
A = 
\mymat{
a_{11} & a_{12} & & \cdots & & a_{1n} \\
a_{21} & a_{22} & & \cdots & & a_{2n} \\
\vdots & \vdots & &        & & \vdots \\
a_{m1} & a_{m2} & & \cdots & & a_{mn} \\
}
\quad\quad\quad\quad\quad
\bx = 
\mymat{
x_1 \\
x_2 \\
\vdots \\
x_n
}
\in \mathbb{R}^n
$$

<br>

It's useful to think of matrix-vector products and matrix-matrix products as combinations of dot products, so we'll look at the dot product first. 

<br>

**Dot Product**: Let $x, y \in \mathbb{R}^n$, then 

<br>

$$
\bx \cdot \by = \bx^T\by = \sum_{k=1}^n x_ky_k = x_1y_1 + x_2y_2 + \cdots + x_ny_n
$$

<br>

If we do an operation count we see that the dot product for two n-length vectors requires $n$ **multiplies** and $(n-1)$ **additions**.  

On typical modern architectures there is only a slight difference between the time it takes to multiply or add two floating-point numbers, so we'll lump these two operations together and refer to them as FLOPS (floating-point operations). 

So the dot product requires about $2n-1$ FLOPS, which for simplicitly we'll round to $2n$ FLOPS. 

<br>

**Matrix-Vector Product**: Consider a real $m \times n$ matrix $A$ and an n-length vector $\bx$. 

Recall that in the row-oriented view of matrix-vector multiplication we computed the $i^{\textrm{th}}$ entry of the resulting m-length vector ${\bf b} = A\bx$ as the dot product the $i^{\textrm{th}}$ row of $A$ with the vector $\bx$. Then the total cost of the mat-vec is 

<br>

$$
m \textrm{ dot products @ } 2n \textrm{ FLOPS each } = 2mn \textrm{ FLOPS}
$$

<br>

Notice that if $A$ is a square $n \times n$ matrix the matrix-vector product is a $2n^2$ FLOP operation. 

Another useful way of computing the number of FLOPS required by an algorithm is by writing the algorithm out in psuedo-code.  Here is one way to compute a row-wise matrix-vector product: 

In [ ]:
b = zeros(m)
for ii=1:m
    for jj=1:n
        b[ii] = b[ii] + A[ii,jj]*x[jj]
    end
end

Note that the outer loop loops over the entries of the vector ${\bf b}$ while the inner loop loops over the $i^{\textrm{th}}$ row of $A$ and computes the dot product with $\bx$. To use the pseudocode to compute the FLOP count we count the number of FLOPS done in the inner-most loop, and then turn the loops into sums.  The operation in the inner-most loop requires 2 FLOPS (1 multiply and 1 add).  So we have 

<br>

$$
\sum_{i=1}^m \sum_{j=1}^n 2 = \sum_{i=1}^m 2n = 2mn 
$$

<br>

**Matrix-Matrix Product**: Consider a real $m \times p$ matrix $A$ and a real $p \times n$ matrix $B$.  The resulting product $AB$ has dimension $m \times n$. 

Recall that the way we thought about matrix-matrix products was that the (i,j)-entry of the product was the dot product of row i of $A$ and col j of $B$.  Since there are $mn$ such entries in $AB$ the cost is 

<br>

$$
mn \textrm{ dot products @ } 2p \textrm{ FLOPS each } = 2mnp \textrm{ FLOPS}
$$

<br>

And if $A$ and $B$ are both square $n \times n$ matrices then the mat-mat product costs $2n^3$ FLOPS. 

If we go the pseudocode route we have 

In [ ]:
C = zeros(m,n)
for ii=1:m
    for jj=1:n
        for kk=1:p
            C[ii,jj] = C[ii,jj] + A[ii,kk]*B[kk,jj]
        end
    end
end

The operation in the inner-most loop again requires 2 FLOPS.  Turning the loops into summations we have 

<br>

$$
\sum_{i=1}^m\sum_{j=1}^n\sum_{k=1}^p 2 = 2mnp \textrm{ FLOPS}
$$

<br>

**Forward Substitution**: Recall that Forward Substition is a method for solving linear systems of the form $L\bx = {\bf b}$ where the matrix $L$ is $n \times n$ and lower triangular. Written out in system form, the system and the solution process look as follows: 

<br>

$$
\begin{array}{rrrrrrr}
\ell_{11}x_1 && && && && = & b_1 \\
\ell_{21}x_1 &+& \ell_{22}x_2 &&&&&& = & b_2 \\
\ell_{31}x_1 &+& \ell_{32}x_2 &+& \ell_{33}x_3 &&&& = & b_n \\
 & & & \vdots & & & & & & \\
\ell_{n1}x_1 &+& \ell_{n2}x_2 &+& \ell_{n3}x_3 &+& \cdots &+& \ell_{nn}x_n = & b_n 
\end{array}
\quad\quad\quad
\begin{array}{l}
x_1 = b_1 / \ell_{11} \\
x_2 = \myp{b_2 - \ell_{21}x_1} / \ell_{22} \\
x_3 = \myp{b_3 - \ell_{31}x_1 - \ell_{32}x_2} / \ell_{33} \\
~\vdots \\
x_n = \myp{b_n - \ell_{n1}x_1 - \cdots - \ell_{n,n-1}x_{n-1}} / \ell_{nn}
\end{array}
$$

<br>

Pseudocode for the Forward Substitution process looks as follows: 

In [ ]:
for ii=1:n
    x[ii] = b[ii]
    for jj=1::ii-1
        x[i] = x[i] - L[ii,jj]*x[jj]
    end
    x[ii] = x[ii]/L[ii,ii]
end

The main brunt of the work occurs inside the two nested loops.  For now we'll ignore assignment and the divide that occur outside of the inner loop.  The operation inside the inner-most loop again requires 2 FLOPS.  Writing the loops as sums we then have 

<br>

$$
\sum_{i=1}^n \sum_{j=1}^{i-1} 2 = 2\sum_{i=1}^n (i-1) = 2 \myp{ \frac{n(n+1)}{2} - n} = n^2-n \approx n^2 \textrm{ FLOPS}
$$

<br>

OK, so the most expensive part of Forward Substitution costs $n^2$ FLOPS.  The assignment operation at the beginning requires $n$ array accesses and $n$ assignments.  The division at the end occurs $n$ times.  Despite the fact that division is between 2 and 8 times as expensive as multiplication and addition, the fact that these things occur only $n$ times is dwarfed by the $n^2$ operations in the inner-most loop.  As a result we say the cost of Forward Substitution is about $n^2$ FLOPS. 

It should be no surprise that Back Substitution also costs about $n^2$ FLOPS, since it requires the same operations as Forward Substition, just in the reverse order. 

<br> 

**LU Decomposition**: Again assume that $A$ is a real $n \times n$ matrix given by 

<br>

$$
A = 
\mymat{
a_{11} & a_{12} & & \cdots & & a_{1n} \\
a_{21} & a_{22} & & \cdots & & a_{2n} \\
\vdots & \vdots & &        & & \vdots \\
a_{m1} & a_{m2} & & \cdots & & a_{mn} \\
}
$$

<br>

The elimination phase of the LU Decomposition proceeds column-by-column and uses the pivot on the diagonal to eliminate the entries in the column below it.  Let's find the cost of just the first phase of elimination where we use the pivot in the (1,1)-position to eliminate everything below it.  

**to eliminate column 1**:

<br>

$$
\begin{array}{llll}
           & \textrm{mults} & \textrm{adds} & \textrm{total} \\
a_{21}: R_2 \leftarrow R_2 - \ell_{21}R_1 & n-1 & n-1 & 2(n-1) \\
a_{31}: R_3 \leftarrow R_3 - \ell_{31}R_1 & n-1 & n-1 & 2(n-1) \\
\quad\vdots & \quad\vdots & \quad\vdots & \quad\vdots \\
a_{n1}: R_n \leftarrow R_n - \ell_{n1}R_1 & n-1 & n-1 & 2(n-1) \\
\end{array}
$$

<br>

We need to do this elimination for each of the $(n-1)$ entries below the pivot in the first column for a grand total of $2(n-1)^2$ FLOPS. 

Note that in counting the number of each operations I'm ignoring a few things.  First, I'm ignoring the computation of the multiplier $\ell{i1}$ which is obtained by dividing the element to eliminate by the pivot. This is a total of $(n-1)$ divides, which is dwarfed by the approximately $n^2$ flops done for the row operations.  I'm also ignoring the the row operations on the actual entries in the first column.  There is no need, for example, to subtract a multiple of $a_{11}$ from $a_{21}$ to zero it out.  We picked the multplier exactly to accomplish this.  

OK, so to eliminate the first column in the matrix requires about $2(n-1)^2$ FLOPS.  Now we need to use the pivot in the (2,2)-position to eliminate everything below it in the second column.  This looks as follows: 

**to eliminate column 1**:

<br>

$$
\begin{array}{llll}
           & \textrm{mults} & \textrm{adds} & \textrm{total} \\
a_{32}: R_3 \leftarrow R_3 - \ell_{32}R_2 & n-2 & n-2 & 2(n-2) \\
a_{42}: R_4 \leftarrow R_4 - \ell_{42}R_2 & n-2 & n-2 & 2(n-2) \\
\quad\vdots & \quad\vdots & \quad\vdots & \quad\vdots \\
a_{n2}: R_n \leftarrow R_n - \ell_{n2}R_2 & n-2 & n-2 & 2(n-2) \\
\end{array}
$$

<br>

Since we have to do these row operations on $(n-2)$ entries in column 2 this gives a grand total of $2(n-2)^2$ FLOPs. 

OK, by now we can see the pattern.  Eliminating the third column is going to require $2(n-3)^2$ flops and so on.  Summing these up we have 

<br>

$$
2(n-1)^2 + 2(n-2)^2 + \cdots 2(1)^2 = 2\sum_{k=1}^n k^2 = 2\myb{\frac{(n-1)n(2n-1)}{6}} \approx \frac{2}{3}n^3 \textrm{ FLOPs}
$$

<br>

**The Integral Trick**: If you have complicated summation formulas that you need to evaluate, and you really only care about the leading order term, you can replace the summation by an integra.  We have 

<br>

$$
2\sum_{k=1}^{n-1}k^2 \approx 2\sum_{k=1}^n \approx 2\int_0^n k^2 dk = \left. \frac{2}{3}k^3 \right|_0^n = \frac{2}{3}n^3
$$

<br>

So the cost of computing the LU Decomposition is approximately $\frac{2}{3}n^3$ FLOPs.  Now we're ready to approximate the cost of solving the linear system $A\bx = {\bf b}$ using LU Decomposition and Forward/Back Substitition.  

<br>

$$
\begin{array}{lcl}
\textrm{LU} & \approx & \frac{2}{3}n^3 \\
\textrm{F Solve} & \approx & n^2 \\
\textrm{B Solve} & \approx & n^2 \\
\hline
\textrm{Total} & \approx & \frac{2}{3}n^3 + 2n^2 \textrm{ FLOPs} \\
\end{array}
$$

<br>

Notice that this is the cost for solving $A\bx = {\bf b}$ for **one** rhs via LU Decomposition.  After computing the decomposition at a cost of roughtly $n^3$, each additional rhs vector that you want to solve with only costs an additional $2n^2$ FLOPs. 

**Matrix Inverse**: Let's now compare this with finding the inverse matrix $A^{-1}$.  Recall that in class we said that the columns of the inverse matrix could be found by solving linear systems of the form 

<br>

$$
A\bx_k = \be_k \textrm{   for   } k=1, \ldots, n
$$

<br>

An efficient strategy for doing this would be to compute the LU Decomposition of $A$ and then use it to solve for the $n$ rhs vectors using Forward and Back substitution.  If we do this the total cost is 

<br>

$$
\frac{2}{3}n^3 + n\myp{2n^2} = \frac{8}{3}n^3 
$$

<br>

So the total cost of finding the inverse is **FOUR TIMES** as expensive as compute the LU Decomposition. 